In [1]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder.appName("cs544")
         .master("spark://boss:7077")
         .config("spark.executor.memory", "512M")
         .config("spark.sql.warehouse.dir", "hdfs://nn:9000/user/hive/warehouse")
         .enableHiveSupport()
         .getOrCreate())

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/09 17:36:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
!hdfs dfs -cp sf.csv hdfs://nn:9000/sf.csv

cp: `hdfs://nn:9000/sf.csv': File exists


In [3]:
df = (spark.read.format("csv")
      .option("header", True)
      .option("inferSchema", True)
      .load("hdfs://nn:9000/sf.csv"))

In [4]:
from pyspark.sql.functions import col, expr
cols = [col(c).alias(c.replace(" ", "_")) for c in df.columns]
df.select(cols).write.format("parquet").mode("overwrite").save("hdfs://nn:9000/sf.parquet")

24/01/09 17:37:12 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [5]:
!hdfs dfs -rm hdfs://nn:9000/sf.csv

Deleted hdfs://nn:9000/sf.csv


In [6]:
df = spark.read.format("parquet").load("hdfs://nn:9000/sf.parquet")

In [7]:
df

DataFrame[Call_Number: int, Unit_ID: string, Incident_Number: int, Call_Type: string, Call_Date: string, Watch_Date: string, Received_DtTm: string, Entry_DtTm: string, Dispatch_DtTm: string, Response_DtTm: string, On_Scene_DtTm: string, Transport_DtTm: string, Hospital_DtTm: string, Call_Final_Disposition: string, Available_DtTm: string, Address: string, City: string, Zipcode_of_Incident: int, Battalion: string, Station_Area: string, Box: string, Original_Priority: string, Priority: string, Final_Priority: int, ALS_Unit: boolean, Call_Type_Group: string, Number_of_Alarms: int, Unit_Type: string, Unit_sequence_in_call_dispatch: int, Fire_Prevention_District: string, Supervisor_District: string, Neighborhooods_-_Analysis_Boundaries: string, RowID: string, case_location: string, Analysis_Neighborhoods: int]

In [8]:
df.limit(3).toPandas()

,Call_Number,Unit_ID,Incident_Number,Call_Type,Call_Date,Watch_Date,Received_DtTm,Entry_DtTm,Dispatch_DtTm,Response_DtTm,...,Call_Type_Group,Number_of_Alarms,Unit_Type,Unit_sequence_in_call_dispatch,Fire_Prevention_District,Supervisor_District,Neighborhooods_-_Analysis_Boundaries,RowID,case_location,Analysis_Neighborhoods
0,201340825,T06,20055834,Other,05/13/2020,05/13/2020,05/13/2020 08:51:30 AM,05/13/2020 08:51:30 AM,05/13/2020 08:54:56 AM,05/13/2020 08:55:10 AM,...,Alarm,1,TRUCK,1,2,8,Mission,201340825-T06,POINT (-122.42251458613262 37.770727334245194),20
1,201241707,FB3,20052084,Outside Fire,05/03/2020,05/03/2020,05/03/2020 01:46:02 PM,05/03/2020 01:46:02 PM,05/03/2020 01:46:11 PM,05/03/2020 01:47:41 PM,...,Alarm,1,SUPPORT,2,1,3,Financial District/South Beach,201241707-FB3,POINT (-122.3946511276673 37.79530574375478),8
2,201083222,AM118,20045948,Medical Incident,04/17/2020,04/17/2020,04/17/2020 09:19:43 PM,04/17/2020 09:19:43 PM,04/17/2020 09:21:10 PM,04/17/2020 09:21:55 PM,...,Non Life-threatening,1,PRIVATE,2,2,6,South of Market,201083222-AM118,POINT (-122.41071806030364 37.778959763358266),34


In [9]:
df.createTempView("calls")

In [10]:
df.createOrReplaceTempView("calls")

In [11]:
spark.sql("SELECT * FROM calls LIMIT 3").toPandas()

,Call_Number,Unit_ID,Incident_Number,Call_Type,Call_Date,Watch_Date,Received_DtTm,Entry_DtTm,Dispatch_DtTm,Response_DtTm,...,Call_Type_Group,Number_of_Alarms,Unit_Type,Unit_sequence_in_call_dispatch,Fire_Prevention_District,Supervisor_District,Neighborhooods_-_Analysis_Boundaries,RowID,case_location,Analysis_Neighborhoods
0,201340825,T06,20055834,Other,05/13/2020,05/13/2020,05/13/2020 08:51:30 AM,05/13/2020 08:51:30 AM,05/13/2020 08:54:56 AM,05/13/2020 08:55:10 AM,...,Alarm,1,TRUCK,1,2,8,Mission,201340825-T06,POINT (-122.42251458613262 37.770727334245194),20
1,201241707,FB3,20052084,Outside Fire,05/03/2020,05/03/2020,05/03/2020 01:46:02 PM,05/03/2020 01:46:02 PM,05/03/2020 01:46:11 PM,05/03/2020 01:47:41 PM,...,Alarm,1,SUPPORT,2,1,3,Financial District/South Beach,201241707-FB3,POINT (-122.3946511276673 37.79530574375478),8
2,201083222,AM118,20045948,Medical Incident,04/17/2020,04/17/2020,04/17/2020 09:19:43 PM,04/17/2020 09:19:43 PM,04/17/2020 09:21:10 PM,04/17/2020 09:21:55 PM,...,Non Life-threatening,1,PRIVATE,2,2,6,South of Market,201083222-AM118,POINT (-122.41071806030364 37.778959763358266),34


In [12]:
spark.sql("SELECT * FROM calls LIMIT 3").explain("formatted")

== Physical Plan ==
CollectLimit (3)
+- * ColumnarToRow (2)
   +- Scan parquet  (1)


(1) Scan parquet 
Output [35]: [Call_Number#228, Unit_ID#229, Incident_Number#230, Call_Type#231, Call_Date#232, Watch_Date#233, Received_DtTm#234, Entry_DtTm#235, Dispatch_DtTm#236, Response_DtTm#237, On_Scene_DtTm#238, Transport_DtTm#239, Hospital_DtTm#240, Call_Final_Disposition#241, Available_DtTm#242, Address#243, City#244, Zipcode_of_Incident#245, Battalion#246, Station_Area#247, Box#248, Original_Priority#249, Priority#250, Final_Priority#251, ALS_Unit#252, Call_Type_Group#253, Number_of_Alarms#254, Unit_Type#255, Unit_sequence_in_call_dispatch#256, Fire_Prevention_District#257, Supervisor_District#258, Neighborhooods_-_Analysis_Boundaries#259, RowID#260, case_location#261, Analysis_Neighborhoods#262]
Batched: true
Location: InMemoryFileIndex [hdfs://nn:9000/sf.parquet]
ReadSchema: struct<Call_Number:int,Unit_ID:string,Incident_Number:int,Call_Type:string,Call_Date:string,Watch_Date:string,Rece

In [13]:
spark.sql("SHOW TABLES").show()

24/01/09 17:39:50 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/01/09 17:39:50 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
24/01/09 17:39:51 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
24/01/09 17:39:51 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore UNKNOWN@172.25.0.3
24/01/09 17:39:51 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


+---------+-------------+-----------+
|namespace|    tableName|isTemporary|
+---------+-------------+-----------+
|  default|calls_by_type|      false|
|  default|       stinky|      false|
|         |        calls|       true|
+---------+-------------+-----------+



In [14]:
spark.sql("""
SELECT * 
FROM calls 
WHERE Call_Type LIKE "Odor%"
LIMIT 5
""").toPandas()

,Call_Number,Unit_ID,Incident_Number,Call_Type,Call_Date,Watch_Date,Received_DtTm,Entry_DtTm,Dispatch_DtTm,Response_DtTm,...,Call_Type_Group,Number_of_Alarms,Unit_Type,Unit_sequence_in_call_dispatch,Fire_Prevention_District,Supervisor_District,Neighborhooods_-_Analysis_Boundaries,RowID,case_location,Analysis_Neighborhoods
0,201121216,E19,20047272,Odor (Strange / Unknown),04/21/2020,04/21/2020,04/21/2020 11:30:07 AM,04/21/2020 11:36:26 AM,04/21/2020 11:37:05 AM,04/21/2020 11:37:21 AM,...,Alarm,1,ENGINE,1,9,7,West of Twin Peaks,201121216-E19,POINT (-122.46804645644951 37.72571348318412),41
1,201363227,KM10,20056915,Odor (Strange / Unknown),05/15/2020,05/15/2020,05/15/2020 07:31:23 PM,05/15/2020 07:36:16 PM,05/15/2020 07:36:53 PM,05/15/2020 07:37:55 PM,...,Alarm,1,PRIVATE,4,7,1,Outer Richmond,201363227-KM10,POINT (-122.47938019994935 37.78130072448512),29
2,201301903,E31,20054576,Odor (Strange / Unknown),05/09/2020,05/09/2020,05/09/2020 03:27:52 PM,05/09/2020 03:30:54 PM,05/09/2020 03:31:15 PM,05/09/2020 03:32:40 PM,...,Alarm,1,ENGINE,1,7,1,Outer Richmond,201301903-E31,POINT (-122.47509052922184 37.78149633273287),29
3,201490091,54,20061869,Odor (Strange / Unknown),05/28/2020,05/27/2020,05/28/2020 01:06:30 AM,05/28/2020 01:06:30 AM,05/28/2020 01:07:33 AM,05/28/2020 01:08:07 AM,...,Alarm,1,MEDIC,1,3,6,Financial District/South Beach,201490091-54,POINT (-122.38902618685223 37.78581422562703),8
4,201453236,E11,20060459,Odor (Strange / Unknown),05/24/2020,05/24/2020,05/24/2020 09:10:36 PM,05/24/2020 09:13:42 PM,05/24/2020 09:14:03 PM,05/24/2020 09:14:52 PM,...,Alarm,1,ENGINE,1,6,8,Noe Valley,201453236-E11,POINT (-122.42538046669073 37.752629190046235),22


In [17]:
spark.sql("""
DROP table stinky
""")

DataFrame[]

In [16]:
%%time
spark.sql("""
SELECT * 
FROM calls 
WHERE Call_Type LIKE "Odor%"
""").write.mode("ignore").saveAsTable("stinky")

24/01/09 17:40:36 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
24/01/09 17:40:36 WARN HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist
24/01/09 17:40:36 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/01/09 17:40:36 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist


CPU times: user 5.66 ms, sys: 1.17 ms, total: 6.83 ms
Wall time: 3.54 s


In [18]:
%%time
spark.sql("""
SELECT * 
FROM calls 
WHERE Call_Type LIKE "Odor%"
""").repartition(3).write.mode("ignore").saveAsTable("stinky")

CPU times: user 3.23 ms, sys: 2.34 ms, total: 5.56 ms
Wall time: 3.09 s


In [19]:
spark.sql("SHOW TABLES").show()

+---------+-------------+-----------+
|namespace|    tableName|isTemporary|
+---------+-------------+-----------+
|  default|calls_by_type|      false|
|  default|       stinky|      false|
|         |        calls|       true|
+---------+-------------+-----------+



In [36]:
!hdfs dfs -ls hdfs://nn:9000/user/hive/warehouse

Found 1 items
drwxr-xr-x   - root supergroup          0 2024-01-08 03:15 hdfs://nn:9000/user/hive/warehouse/stinky


In [20]:
!hdfs dfs -ls hdfs://nn:9000/user/hive/warehouse/stinky

Found 4 items
-rw-r--r--   3 root supergroup          0 2024-01-09 17:41 hdfs://nn:9000/user/hive/warehouse/stinky/_SUCCESS
-rw-r--r--   3 root supergroup     539241 2024-01-09 17:41 hdfs://nn:9000/user/hive/warehouse/stinky/part-00000-061aca15-aaef-4851-8bf1-eb40b59e8e7c-c000.snappy.parquet
-rw-r--r--   3 root supergroup     534814 2024-01-09 17:41 hdfs://nn:9000/user/hive/warehouse/stinky/part-00001-061aca15-aaef-4851-8bf1-eb40b59e8e7c-c000.snappy.parquet
-rw-r--r--   3 root supergroup     535977 2024-01-09 17:41 hdfs://nn:9000/user/hive/warehouse/stinky/part-00002-061aca15-aaef-4851-8bf1-eb40b59e8e7c-c000.snappy.parquet


In [21]:
%%time
spark.sql("""
SELECT * 
FROM calls 
WHERE Call_Type LIKE "Odor%"
""").rdd.getNumPartitions()

CPU times: user 1.28 ms, sys: 904 µs, total: 2.18 ms
Wall time: 52.2 ms


6

In [22]:
%%time
spark.sql("""
SELECT * 
FROM stinky
""").rdd.getNumPartitions()

CPU times: user 1.74 ms, sys: 0 ns, total: 1.74 ms
Wall time: 70.9 ms


2

In [23]:
spark.table("calls")

DataFrame[Call_Number: int, Unit_ID: string, Incident_Number: int, Call_Type: string, Call_Date: string, Watch_Date: string, Received_DtTm: string, Entry_DtTm: string, Dispatch_DtTm: string, Response_DtTm: string, On_Scene_DtTm: string, Transport_DtTm: string, Hospital_DtTm: string, Call_Final_Disposition: string, Available_DtTm: string, Address: string, City: string, Zipcode_of_Incident: int, Battalion: string, Station_Area: string, Box: string, Original_Priority: string, Priority: string, Final_Priority: int, ALS_Unit: boolean, Call_Type_Group: string, Number_of_Alarms: int, Unit_Type: string, Unit_sequence_in_call_dispatch: int, Fire_Prevention_District: string, Supervisor_District: string, Neighborhooods_-_Analysis_Boundaries: string, RowID: string, case_location: string, Analysis_Neighborhoods: int]

In [24]:
spark.table("stinky")

DataFrame[Call_Number: int, Unit_ID: string, Incident_Number: int, Call_Type: string, Call_Date: string, Watch_Date: string, Received_DtTm: string, Entry_DtTm: string, Dispatch_DtTm: string, Response_DtTm: string, On_Scene_DtTm: string, Transport_DtTm: string, Hospital_DtTm: string, Call_Final_Disposition: string, Available_DtTm: string, Address: string, City: string, Zipcode_of_Incident: int, Battalion: string, Station_Area: string, Box: string, Original_Priority: string, Priority: string, Final_Priority: int, ALS_Unit: boolean, Call_Type_Group: string, Number_of_Alarms: int, Unit_Type: string, Unit_sequence_in_call_dispatch: int, Fire_Prevention_District: string, Supervisor_District: string, Neighborhooods_-_Analysis_Boundaries: string, RowID: string, case_location: string, Analysis_Neighborhoods: int]

# Grouping, other Queries

In [57]:
# what areas are in SF?

In [25]:
df.withColumnRenamed("Neighborhooods_-_Analysis_Boundaries","area").createOrReplaceTempView("calls")

In [26]:
spark.table("calls")

DataFrame[Call_Number: int, Unit_ID: string, Incident_Number: int, Call_Type: string, Call_Date: string, Watch_Date: string, Received_DtTm: string, Entry_DtTm: string, Dispatch_DtTm: string, Response_DtTm: string, On_Scene_DtTm: string, Transport_DtTm: string, Hospital_DtTm: string, Call_Final_Disposition: string, Available_DtTm: string, Address: string, City: string, Zipcode_of_Incident: int, Battalion: string, Station_Area: string, Box: string, Original_Priority: string, Priority: string, Final_Priority: int, ALS_Unit: boolean, Call_Type_Group: string, Number_of_Alarms: int, Unit_Type: string, Unit_sequence_in_call_dispatch: int, Fire_Prevention_District: string, Supervisor_District: string, area: string, RowID: string, case_location: string, Analysis_Neighborhoods: int]

In [27]:
spark.sql("""
SELECT DISTINCT area
FROM calls
""").collect()

[Row(area='Inner Sunset'),
 Row(area='Haight Ashbury'),
 Row(area='Lincoln Park'),
 Row(area='Japantown'),
 Row(area='None'),
 Row(area='North Beach'),
 Row(area='Lone Mountain/USF'),
 Row(area='Western Addition'),
 Row(area='Bernal Heights'),
 Row(area='Mission Bay'),
 Row(area='Hayes Valley'),
 Row(area='Financial District/South Beach'),
 Row(area='Lakeshore'),
 Row(area='Bayview Hunters Point'),
 Row(area='Visitacion Valley'),
 Row(area='Inner Richmond'),
 Row(area='Nob Hill'),
 Row(area='Oceanview/Merced/Ingleside'),
 Row(area='Outer Richmond'),
 Row(area='Treasure Island'),
 Row(area='Chinatown'),
 Row(area='Mission'),
 Row(area='Excelsior'),
 Row(area='Twin Peaks'),
 Row(area='Seacliff'),
 Row(area='Sunset/Parkside'),
 Row(area='Presidio Heights'),
 Row(area='Portola'),
 Row(area='Golden Gate Park'),
 Row(area='Glen Park'),
 Row(area='McLaren Park'),
 Row(area='Presidio'),
 Row(area='Tenderloin'),
 Row(area='Potrero Hill'),
 Row(area='Outer Mission'),
 Row(area='Castro/Upper Mark

In [65]:
# how many calls are there per area?
spark.sql("""
SELECT DISTINCT area, COUNT(*) AS count
FROM calls
GROUP BY area
ORDER BY count DESC
""").toPandas()

,area,count
0,Tenderloin,809041
1,South of Market,583631
2,Mission,543289
3,Financial District/South Beach,402331
4,Bayview Hunters Point,326423
5,Sunset/Parkside,233135
6,Western Addition,222759
7,Nob Hill,198957
8,Outer Richmond,159714
9,Hayes Valley,148946


In [30]:
# how many calls are per group/type
spark.sql("""
SELECT Call_Type_Group, Call_type, COUNT(*) AS count
FROM calls
GROUP BY Call_Type_group, Call_type
ORDER BY Call_Type_group DESC, count DESC
""").toPandas()

,Call_Type_Group,Call_type,count
0,Potentially Life-Threatening,Medical Incident,1448636
1,Potentially Life-Threatening,Traffic Collision,86830
2,Potentially Life-Threatening,Other,1763
3,Potentially Life-Threatening,Water Rescue,93
4,Potentially Life-Threatening,Structure Fire,16
...,...,...,...
73,None,Suspicious Package,290
74,None,Confined Space / Structure Collapse,286
75,None,Marine Fire,222
76,None,Train / Rail Fire,74


In [69]:
# for each group, what perc of calls are represented by the biggest type?
spark.sql("""
SELECT Call_Type_Group, MAX(count)/SUM(count)
FROM (
    SELECT Call_Type_Group, Call_type, COUNT(*) AS count
    FROM calls
    GROUP BY Call_Type_group, Call_type
)
GROUP BY Call_Type_group
""").toPandas()

,Call_Type_Group,(max(count) / sum(count))
0,Alarm,0.486623
1,None,0.632953
2,Potentially Life-Threatening,0.942302
3,Non Life-threatening,0.924175
4,Fire,0.383007


In [33]:
# same question, but with Spark DataFrame API
# how many calls are per group/type
(spark.table("calls")
 .groupby("Call_Type_Group", "Call_type")
 .count()
 .orderBy("count")
).toPandas()

,Call_Type_Group,Call_type,count
0,Fire,Administrative,1
1,Alarm,Oil Spill,5
2,None,Lightning Strike (Investigation),9
3,Fire,Train / Rail Fire,10
4,Alarm,Vehicle Fire,13
...,...,...,...
73,Alarm,Alarms,366116
74,None,Structure Fire,416250
75,Non Life-threatening,Medical Incident,716653
76,Potentially Life-Threatening,Medical Incident,1448636


In [34]:
# same question, but with Spark DataFrame API
# how many calls are per group/type
from pyspark.sql.functions import *
(spark.table("calls")
 .groupby("Call_Type_Group", "Call_type")
 .count()
 .orderBy(desc("count"))
).toPandas()

,Call_Type_Group,Call_type,count
0,None,Medical Incident,1783402
1,Potentially Life-Threatening,Medical Incident,1448636
2,Non Life-threatening,Medical Incident,716653
3,None,Structure Fire,416250
4,Alarm,Alarms,366116
...,...,...,...
73,Alarm,Vehicle Fire,13
74,Fire,Train / Rail Fire,10
75,None,Lightning Strike (Investigation),9
76,Alarm,Oil Spill,5


In [36]:
# same question, but with Spark DataFrame API
# for each group, what perc of calls are represented by the biggest type?

(spark.table("calls")
 .groupby("Call_Type_Group", "Call_type")
 .count()
 .groupby("Call_Type_Group")
 .agg(expr("MAX(count) / SUM(count)").alias("perc"))
 .orderBy("perc")
).toPandas()

,Call_Type_Group,perc
0,Fire,0.383007
1,Alarm,0.486623
2,None,0.632953
3,Non Life-threatening,0.924175
4,Potentially Life-Threatening,0.942302


In [37]:
(spark.table("calls")
 .groupby("Call_Type_Group", "Call_type")
 .count()
 .groupby("Call_Type_Group")
 .agg(expr("MAX(count) / SUM(count)").alias("perc"))
 .orderBy("perc")
).explain("formatted")

== Physical Plan ==
AdaptiveSparkPlan (10)
+- Sort (9)
   +- Exchange (8)
      +- HashAggregate (7)
         +- Exchange (6)
            +- HashAggregate (5)
               +- HashAggregate (4)
                  +- Exchange (3)
                     +- HashAggregate (2)
                        +- Scan parquet  (1)


(1) Scan parquet 
Output [2]: [Call_Type#231, Call_Type_Group#253]
Batched: true
Location: InMemoryFileIndex [hdfs://nn:9000/sf.parquet]
ReadSchema: struct<Call_Type:string,Call_Type_Group:string>

(2) HashAggregate
Input [2]: [Call_Type#231, Call_Type_Group#253]
Keys [2]: [Call_Type_Group#253, Call_type#231]
Functions [1]: [partial_count(1)]
Aggregate Attributes [1]: [count#1458L]
Results [3]: [Call_Type_Group#253, Call_type#231, count#1459L]

(3) Exchange
Input [3]: [Call_Type_Group#253, Call_type#231, count#1459L]
Arguments: hashpartitioning(Call_Type_Group#253, Call_type#231, 200), ENSURE_REQUIREMENTS, [plan_id=1039]

(4) HashAggregate
Input [3]: [Call_Type_Group#253, C

In [38]:
spark.table("calls")

DataFrame[Call_Number: int, Unit_ID: string, Incident_Number: int, Call_Type: string, Call_Date: string, Watch_Date: string, Received_DtTm: string, Entry_DtTm: string, Dispatch_DtTm: string, Response_DtTm: string, On_Scene_DtTm: string, Transport_DtTm: string, Hospital_DtTm: string, Call_Final_Disposition: string, Available_DtTm: string, Address: string, City: string, Zipcode_of_Incident: int, Battalion: string, Station_Area: string, Box: string, Original_Priority: string, Priority: string, Final_Priority: int, ALS_Unit: boolean, Call_Type_Group: string, Number_of_Alarms: int, Unit_Type: string, Unit_sequence_in_call_dispatch: int, Fire_Prevention_District: string, Supervisor_District: string, area: string, RowID: string, case_location: string, Analysis_Neighborhoods: int]

### WINDOW FUNCTIONS in SQL

In [ ]:
# # what are the 3 SMALLEST call number for each area
# spark.sql("""
# SELECT area, Call_Number, WINDOW_FUNC() OVER (PARTITIONING)
# FROM calls
# """)

In [44]:
# exploring by fti
spark.sql("""
SELECT DISTINCT Priority
FROM calls
""").toPandas()

,Priority
0,3
1,E
2,B
3,C
4,A
5,1
6,I
7,2
8,T
9,None


In [48]:
# exploring by fti
spark.sql("""
SELECT Call_Number, Priority
FROM calls
ORDER BY Call_Number
""").limit(10).toPandas()

,Call_Number,Priority
0,1030101,3
1,1030101,3
2,1030104,3
3,1030104,3
4,1030106,3
5,1030107,3
6,1030107,3
7,1030107,3
8,1030108,3
9,1030108,3


In [49]:
# exploring by fti
spark.sql("""
SELECT Call_Number, SUM(Priority) OVER (ORDER BY Call_Number) AS running_total
FROM calls
""").limit(10).toPandas()

24/01/09 18:16:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 18:16:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 18:16:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 18:16:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/01/09 18:16:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


,Call_Number,running_total
0,1030101,6.0
1,1030101,6.0
2,1030104,12.0
3,1030104,12.0
4,1030106,15.0
5,1030107,24.0
6,1030107,24.0
7,1030107,24.0
8,1030108,30.0
9,1030108,30.0


In [53]:
# exploring by fti
spark.sql("""
SELECT area, Call_Number, Priority
FROM calls
ORDER BY area, Call_Number
""").limit(10).toPandas()

,area,Call_Number,Priority
0,Bayview Hunters Point,1030119,1
1,Bayview Hunters Point,1040031,1
2,Bayview Hunters Point,1040078,1
3,Bayview Hunters Point,1040112,3
4,Bayview Hunters Point,1040112,3
5,Bayview Hunters Point,1040139,3
6,Bayview Hunters Point,1040139,3
7,Bayview Hunters Point,1040145,3
8,Bayview Hunters Point,1040145,3
9,Bayview Hunters Point,1040169,3


In [50]:
# exploring by fti
spark.sql("""
SELECT area, Call_Number, SUM(Priority) OVER (PARTITION BY area ORDER BY Call_Number) AS running_total
FROM calls
ORDER BY area
""").limit(10).toPandas()

,area,Call_Number,running_total
0,Bayview Hunters Point,1030119,1.0
1,Bayview Hunters Point,1040031,2.0
2,Bayview Hunters Point,1040078,3.0
3,Bayview Hunters Point,1040112,9.0
4,Bayview Hunters Point,1040112,9.0
5,Bayview Hunters Point,1040139,15.0
6,Bayview Hunters Point,1040139,15.0
7,Bayview Hunters Point,1040145,21.0
8,Bayview Hunters Point,1040145,21.0
9,Bayview Hunters Point,1040169,24.0


In [59]:
# exploring by fti
spark.sql("""
SELECT area, Call_Number, COUNT(*) OVER (PARTITION BY area ORDER BY Call_Number) AS count
FROM calls
ORDER BY area
""").limit(10).toPandas()

,area,Call_Number,count
0,Bayview Hunters Point,1030119,1
1,Bayview Hunters Point,1040031,2
2,Bayview Hunters Point,1040078,3
3,Bayview Hunters Point,1040112,5
4,Bayview Hunters Point,1040112,5
5,Bayview Hunters Point,1040139,7
6,Bayview Hunters Point,1040139,7
7,Bayview Hunters Point,1040145,9
8,Bayview Hunters Point,1040145,9
9,Bayview Hunters Point,1040169,10


In [72]:
# exploring by fti
spark.sql("""
SELECT area, Call_Number, row_number() OVER (PARTITION BY area ORDER BY Call_Number) AS row_num
FROM calls
ORDER BY Call_Number, area
""").limit(50).toPandas()

,area,Call_Number,row_num
0,Sunset/Parkside,1030101,1
1,Sunset/Parkside,1030101,2
2,Sunset/Parkside,1030104,3
3,Sunset/Parkside,1030104,4
4,Tenderloin,1030106,1
5,Tenderloin,1030107,2
6,Tenderloin,1030107,3
7,Tenderloin,1030107,4
8,Financial District/South Beach,1030108,1
9,Financial District/South Beach,1030108,2


In [87]:
# what are the 3 SMALLEST call number for each area
spark.sql("""
SELECT area, Call_Number, row_number() OVER (PARTITION BY area ORDER BY Call_Number ASC) AS rownum
FROM calls
""").limit(10).toPandas()

,area,Call_Number,rownum
0,Bayview Hunters Point,1030119,1
1,Bayview Hunters Point,1040031,2
2,Bayview Hunters Point,1040078,3
3,Bayview Hunters Point,1040112,4
4,Bayview Hunters Point,1040112,5
5,Bayview Hunters Point,1040139,6
6,Bayview Hunters Point,1040139,7
7,Bayview Hunters Point,1040145,8
8,Bayview Hunters Point,1040145,9
9,Bayview Hunters Point,1040169,10


In [73]:
# what are the 3 SMALLEST call number for each area
spark.sql("""
SELECT area, Call_Number, row_number() OVER (PARTITION BY area ORDER BY Call_Number ASC) AS rownum
FROM calls
""").where("rownum <= 3").toPandas()

,area,Call_Number,rownum
0,Bayview Hunters Point,1030119,1
1,Bayview Hunters Point,1040031,2
2,Bayview Hunters Point,1040078,3
3,Bernal Heights,1040089,1
4,Bernal Heights,1040245,2
...,...,...,...
121,West of Twin Peaks,1040020,2
122,West of Twin Peaks,1040115,3
123,Western Addition,1030128,1
124,Western Addition,1030128,2
